This time I am going to ignore the outliers.

In [1]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import (
    FunctionTransformer,
    PowerTransformer,
    MinMaxScaler,
    OneHotEncoder,
    Binarizer,
    StandardScaler
)


from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.decomposition import PCA


import mlflow
import mlflow.sklearn

from matplotlib import pyplot as plt
import seaborn as sns
import optuna

In [2]:
dataset_train = pd.read_csv('../data/processed/dataset_train.csv', index_col=0)
dataset_test = pd.read_csv('../data/processed/dataset_test.csv', index_col=0)

In [3]:
# column names
cols_event_embeddings = [f'event_feat_{i}' for i in range(768)]
cols_event_dummies = ['BACK STRAIN', 'CLEANING INSTRUMENTS', 'CLEANING LEFT SHOULDER SPLINTER', 'CUT WITH KNIFE', 'DEALING CARDS RIGHT TENDON SYNOVITIS', 'FELL', 'FELL FROM LADDER', 'FELL OFF LADDER', 'FRACTURED LEFT WRIST FRACTURE', 'GRINDING FOREIGN BODY', 'GRINDING STEEL BEAM INFECTION', 'HIT AIR HOSE', 'HIT ELBOW', 'HIT FLOOR SHEET', 'INJURY', 'INJURY LIFTING', 'INJURY LIFTING STRAIN', 'JAMMED RIGHT HAND PUNCTURE', 'KNIFE LACERATION', 'KNIFE SLIPPED', 'LACERATION', 'LEFT LEG SNAKE BITE', 'LIFTING', 'LIFTING BOXES', 'LIFTING PATIENT', 'LIFTING TYRES', 'LOWER BACK STRAIN', 'MOTOR VEHICLE COLLISION', 'PREPARING PIPES FOR PAINTING PUNCTURE', 'PULLING CABLES', 'REDBACK SPIDER BITE', 'SLIPPED AND FELL', 'SLIPPED ON FLOOR', 'SLIPPED ON ROLLER', 'SLIPPED ON STAIRS', 'SLIPPED ON WET FLOOR', 'SLIPPED USING LATHE', 'SORTING ALUMINIUM BARS STRAIN', 'SPRAINED RIGHT ANKLE FRACTURE', 'STRAIN', 'STRAIN LIFTING PARTS', 'STRAIN LIFTING STRAIN', 'STRUCK KNIFE LACERATED', 'STRUCK KNIFE LACERATION', 'STRUCK PALLET', 'STRUCK WITH KNIFE', 'USING AIR HOSE STRAIN', 'USING DRILL']
cols_body_parts_embeddings = [f'body_feat_{i}' for i in range(768)]

In [4]:
(dataset_train['UltimateIncurredClaimCost'] > 5e5).sum()

np.int64(13)

In [ ]:
# removing values over 500000 will erase 13 rows
dataset_train = dataset_train[dataset_train['UltimateIncurredClaimCost'] < 5e5]

In [6]:
def train_cv(model, X, y, metric = 'rmse', kfold = 5):

    if metric != 'rmse':
        raise NotImplemented

    scores = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')

    mse_scores = -scores
    rmse_scores = np.sqrt(mse_scores)

    return rmse_scores.mean(), rmse_scores.std()

In [7]:
log_transformer = Pipeline([
    ('log', FunctionTransformer(np.log, validate=False)),
    ('scaler', StandardScaler()) 
])

# gender to bool 
def gender_to_bool(gender_column):
    """
    - Fill missing values with 'M'
    - Replace 'U' with 'M'
    - Return a boolean-ish column: 1 for 'M', 0 otherwise
    """
    g = pd.Series(gender_column.squeeze(), dtype=str).fillna('M').replace('U', 'M')
    is_male = (g == 'M').astype(int)
    return is_male.values.reshape(-1, 1)

gender_transformer = FunctionTransformer(gender_to_bool, validate=False)

# hours worked per week -> to buckets -> to one_hot
def bucket_hours_worked(dtt_array):
    return pd.cut(
        dtt_array.squeeze(), 
        bins=[-np.inf, 37, 41, np.inf],
        labels=["<=37", "37-41", ">41"]
    ).astype(str).values.reshape(-1, 1)

hours_worked_bucketer = FunctionTransformer(bucket_hours_worked, validate=False)
hours_worked_encoder = OneHotEncoder(drop='first')
hours_worked_pipeline = Pipeline([
    ('bucketizer', hours_worked_bucketer),
    ('encoder', hours_worked_encoder)
])

# DaysToReport (DateReported - DateTimeOfAccident) -> to buckets -> to one_hot
def bucket_days_to_report(dtt_array):
    return pd.cut(
        dtt_array.squeeze(), 
        bins=[-np.inf, 80, 300, 500, np.inf],
        labels=["<=80", "80-300", "300-500", ">500"]
    ).astype(str).values.reshape(-1, 1)

days_to_report_bucketer = FunctionTransformer(bucket_days_to_report, validate=False)
days_to_report_encoder = OneHotEncoder(drop='first')
days_to_report_pipeline = Pipeline([
    ('bucketizer', days_to_report_bucketer),
    ('encoder', days_to_report_encoder)
])

# DaysWorkedPerWeek -> 1 if equals 5, 0 in any other case 
def days_worked_binarize(days_array):
    # Ensure we handle arrays or DataFrames by squeezing to 1D
    days = days_array.squeeze()
    binarized = (days == 5).astype(int)
    # Return as 2D array: (n_samples x 1)
    return binarized.values.reshape(-1, 1) if isinstance(days, pd.Series) else binarized.reshape(-1, 1)

days_worked_transformer = FunctionTransformer(days_worked_binarize, validate=False)

In [8]:
preprocessor = ColumnTransformer(
    transformers=[
        ('log_inc', log_transformer, ['InitialIncurredCalimsCost', 'WeeklyWages']),
        ('minmax_scaler', MinMaxScaler(), ['Age', 'YearAccident']),
        ('gender_bool', gender_transformer, ['Gender']),
        ('hww_bool_onehot', hours_worked_pipeline, ['HoursWorkedPerWeek']),
        ('dtt_bool_onehot', days_to_report_pipeline, ['DaysToReport']),
        ('has_dependent_bool', Binarizer(threshold=0), ['DependentChildren']),
        ('worked_five_days_bool', days_worked_transformer, ['DaysWorkedPerWeek']),
        ('onehot', OneHotEncoder(drop='first'), ['MaritalStatus', 'PartTimeFullTime']),
        ('pca_event_embeddings', PCA(n_components=250), cols_event_embeddings),
        ('passthrough_event_dummies', 'passthrough', cols_event_dummies),
        ('pca_body_parts_embeddings', PCA(n_components=80), cols_body_parts_embeddings),
        ('std_scaler', StandardScaler(), ['event_median_cost'])
    ],
    remainder='drop'
)

In [9]:
regressor_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('linear_model', XGBRegressor())
])

model = TransformedTargetRegressor(
    regressor=regressor_pipeline,
    transformer=PowerTransformer(method='box-cox', standardize=False)
)

In [10]:

scores = cross_val_score(
    model, 
    dataset_train, 
    dataset_train['UltimateIncurredClaimCost'], 
    cv=5, 
    scoring='neg_mean_squared_error'
)
mse_scores = -scores
rmse_scores = np.sqrt(mse_scores)

print(f"CV MSE:  {mse_scores.mean():.3f}  (+/- {mse_scores.std():.3f})")
print(f"CV RMSE: {rmse_scores.mean():.3f}  (+/- {rmse_scores.std():.3f})")

CV MSE:  499651363.017  (+/- 39383780.495)
CV RMSE: 22335.797  (+/- 873.800)


As expected, even the out of the box version of xgboost has better error than previous attempts. 

In [11]:
X_train = dataset_train.drop(columns=['UltimateIncurredClaimCost'])
y_train = dataset_train['UltimateIncurredClaimCost']

def run_experiment(
        experiment_name, 
        run_name, 
        regressor_object, 
        kfold=5, 
        save_model=True
    ):
    """
    Runs an ML experiment with cross-validation and logs metrics to MLflow,
    tracking overall and segmented performance for targets above and below 100,000.

    Parameters:
    - experiment_name (str): Name of the MLflow experiment.
    - run_name (str): Name of the MLflow run.
    - regressor_object: The regression model.
    - kfold (int): Number of folds for cross-validation.
    - save_model (bool): Whether to save the trained model in MLflow.

    Logs:
    - Overall CV MSE, RMSE, and MAE
    - MSE, RMSE, and MAE for y_true > 100,000
    - MSE, RMSE, and MAE for y_true ≤ 100,000
    """

    # Define pipeline with preprocessing and model
    regressor_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('linear_model', regressor_object)
    ])

    model = TransformedTargetRegressor(
        regressor=regressor_pipeline,
        transformer=PowerTransformer(method='box-cox', standardize=False)
    )

    # Set MLflow experiment
    mlflow.set_experiment(experiment_name)

    with mlflow.start_run(run_name=run_name):
        # Log hyperparameters
        params = model.get_params(deep=True)
        for param_name, param_value in params.items():
            if isinstance(param_value, (str, int, float, bool, type(None))):
                mlflow.log_param(param_name, param_value)
            else:
                mlflow.log_param(param_name, str(param_value)[:5000])
        
        mlflow.log_param("n_features_in", X_train.shape[1])

        # Perform cross-validation with predictions
        y_pred = cross_val_predict(model, X_train, y_train, cv=kfold)

        # Compute overall errors
        overall_mse = mean_squared_error(y_train, y_pred)
        overall_rmse = np.sqrt(overall_mse)
        overall_mae = mean_absolute_error(y_train, y_pred)

        # Compute errors for high and low y_true
        high_mask = y_train > 100000

        high_mse = mean_squared_error(y_train[high_mask], y_pred[high_mask])
        high_rmse = np.sqrt(high_mse)
        high_mae = mean_absolute_error(y_train[high_mask], y_pred[high_mask])
        
        low_mse = mean_squared_error(y_train[~high_mask], y_pred[~high_mask])
        low_rmse = np.sqrt(low_mse)
        low_mae = mean_absolute_error(y_train[~high_mask], y_pred[~high_mask])
    
        print(f"RMSE Overall:  {overall_rmse:.3f} | MAE: {overall_mae:.3f}")
        print(f"RMSE for low y_true (<= 100k): {low_rmse:.3f} | MAE: {low_mae:.3f}")
        print(f"RMSE for high y_true (> 100k): {high_rmse:.3f} | MAE: {high_mae:.3f}")
        

        # Log overall metrics
        mlflow.log_metric("cv_mse", overall_mse)
        mlflow.log_metric("cv_rmse", overall_rmse)
        mlflow.log_metric("cv_mae", overall_mae)

        # Log segmented metrics
        mlflow.log_metric("cv_mse_high", high_mse)
        mlflow.log_metric("cv_rmse_high", high_rmse)
        mlflow.log_metric("cv_mae_high", high_mae)
        mlflow.log_metric("cv_mse_low", low_mse)
        mlflow.log_metric("cv_rmse_low", low_rmse)
        mlflow.log_metric("cv_mae_low", low_mae)

        # Fit final model on full dataset
        if save_model:
            model.fit(X_train, y_train)
            mlflow.sklearn.log_model(model, artifact_path="models")
    
    return overall_rmse


In [12]:

def objective(trial):
    """Objective function for Optuna to optimize XGBoost hyperparameters."""
    
    # Suggest hyperparameters to optimize
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 10.0),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
    }

    rmse = run_experiment(
        'Optuna XGBoost - no outliers',
        'XGBoost optimization',
        XGBRegressor(**params),
        kfold=5,
        save_model=False
    )

    return rmse


In [13]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=35)

# Print the best result
print(f"Best trial: {study.best_trial.params}")


[I 2025-03-02 18:02:57,614] A new study created in memory with name: no-name-a5fd74d6-a3b7-4974-9e03-c000325bda38
[I 2025-03-02 18:03:16,067] Trial 0 finished with value: 22068.84161051241 and parameters: {'n_estimators': 600, 'max_depth': 7, 'learning_rate': 0.05530492523604442, 'min_child_weight': 5, 'subsample': 0.8945426311715513, 'colsample_bytree': 0.56843185328282, 'reg_alpha': 0.5536031355636127, 'reg_lambda': 7.666209318263942, 'gamma': 3.953700546676619}. Best is trial 0 with value: 22068.84161051241.


RMSE Overall:  22068.842 | MAE: 5462.042
RMSE for low y_true (<= 100k): 9149.135 | MAE: 3173.861
RMSE for high y_true (> 100k): 150724.319 | MAE: 131583.146


[I 2025-03-02 18:03:34,217] Trial 1 finished with value: 21967.684197337636 and parameters: {'n_estimators': 800, 'max_depth': 6, 'learning_rate': 0.10651476051010814, 'min_child_weight': 2, 'subsample': 0.5872701543423388, 'colsample_bytree': 0.8842607026647225, 'reg_alpha': 0.3247646639166898, 'reg_lambda': 8.731439600303757, 'gamma': 2.3993869122683216}. Best is trial 1 with value: 21967.684197337636.


RMSE Overall:  21967.684 | MAE: 5418.965
RMSE for low y_true (<= 100k): 9037.466 | MAE: 3142.340
RMSE for high y_true (> 100k): 150265.671 | MAE: 130903.132


[I 2025-03-02 18:04:03,604] Trial 2 finished with value: 22043.37413065177 and parameters: {'n_estimators': 900, 'max_depth': 9, 'learning_rate': 0.0244694316952387, 'min_child_weight': 1, 'subsample': 0.5867588583826903, 'colsample_bytree': 0.8456542095932117, 'reg_alpha': 0.2137328826130912, 'reg_lambda': 6.119093515902776, 'gamma': 2.613877294731091}. Best is trial 1 with value: 21967.684197337636.


...

RMSE Overall:  21854.298 | MAE: 5477.005
RMSE for low y_true (<= 100k): 9287.281 | MAE: 3247.324
RMSE for high y_true (> 100k): 148487.742 | MAE: 128373.640


[I 2025-03-02 18:13:07,893] Trial 33 finished with value: 21835.750209636237 and parameters: {'n_estimators': 800, 'max_depth': 4, 'learning_rate': 0.030232416594797168, 'min_child_weight': 5, 'subsample': 0.9348382433357434, 'colsample_bytree': 0.6783730222530824, 'reg_alpha': 0.7620160424974837, 'reg_lambda': 6.173470917476139, 'gamma': 0.8172031120662108}. Best is trial 33 with value: 21835.750209636237.


RMSE Overall:  21835.750 | MAE: 5453.946
RMSE for low y_true (<= 100k): 9248.083 | MAE: 3224.448
RMSE for high y_true (> 100k): 148469.456 | MAE: 128340.562


[I 2025-03-02 18:13:35,321] Trial 34 finished with value: 21833.27461769771 and parameters: {'n_estimators': 800, 'max_depth': 4, 'learning_rate': 0.030891804250286765, 'min_child_weight': 5, 'subsample': 0.9432993103831395, 'colsample_bytree': 0.6808937148959778, 'reg_alpha': 0.9409491829357446, 'reg_lambda': 6.2499627510090665, 'gamma': 0.8376028736074165}. Best is trial 34 with value: 21833.27461769771.


RMSE Overall:  21833.275 | MAE: 5450.320
RMSE for low y_true (<= 100k): 9232.365 | MAE: 3219.282
RMSE for high y_true (> 100k): 148502.941 | MAE: 128421.808
Best trial: {'n_estimators': 800, 'max_depth': 4, 'learning_rate': 0.030891804250286765, 'min_child_weight': 5, 'subsample': 0.9432993103831395, 'colsample_bytree': 0.6808937148959778, 'reg_alpha': 0.9409491829357446, 'reg_lambda': 6.2499627510090665, 'gamma': 0.8376028736074165}


In [21]:
import optuna.visualization as vis

fig = vis.plot_optimization_history(study)
fig.show()


In [22]:
fig = vis.plot_param_importances(study)
fig.show()


In [24]:
best_params = study.best_trial.params


regressor_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('linear_model', XGBRegressor(**best_params))
])

model = TransformedTargetRegressor(
    regressor=regressor_pipeline,
    transformer=PowerTransformer(method='box-cox', standardize=False)
)

In [25]:
model.fit(X_train, y_train)

TransformedTargetRegressor(regressor=Pipeline(steps=[('preprocessor',
                                                      ColumnTransformer(transformers=[('log_inc',
                                                                                       Pipeline(steps=[('log',
                                                                                                        FunctionTransformer(func=<ufunc 'log'>)),
                                                                                                       ('scaler',
                                                                                                        StandardScaler())]),
                                                                                       ['InitialIncurredCalimsCost',
                                                                                        'WeeklyWages']),
                                                                                      ('minmax_scaler',
                                                                                       MinMaxScaler(),
                                                                                       ['Age',
                                                                                        'YearAccident']),
                                                                                      ('gender_bool',
                                                                                       FunctionTransformer(func=<fun...
                                                                   learning_rate=0.030891804250286765,
                                                                   max_bin=None,
                                                                   max_cat_threshold=None,
                                                                   max_cat_to_onehot=None,
                                                                   max_delta_step=None,
                                                                   max_depth=4,
                                                                   max_leaves=None,
                                                                   min_child_weight=5,
                                                                   missing=nan,
                                                                   monotone_constraints=None,
                                                                   multi_strategy=None,
                                                                   n_estimators=800,
                                                                   n_jobs=None,
                                                                   num_parallel_tree=None,
                                                                   random_state=None, ...))]),
                           transformer=PowerTransformer(method='box-cox',
                                                        standardize=False))

In [26]:
y_test_pred = model.predict(dataset_test)
y_test_pred

array([ 5994.1753 ,  2736.7083 , 19513.99   , ...,  6074.552  ,
        7215.8286 ,   417.86932], shape=(36000,), dtype=float32)

In [27]:
submission_df = pd.DataFrame(zip(dataset_test.index, y_test_pred), columns=['ClaimNumber', 'UltimateIncurredClaimCost'])

print(submission_df.shape)
submission_df.head()

(36000, 2)


,ClaimNumber,UltimateIncurredClaimCost
0,WC8145235,5994.175293
1,WC2005111,2736.708252
2,WC6899143,19513.990234
3,WC5502023,359.316650
4,WC4785156,3069.980469


In [ ]:
submission_df.to_csv('../data/output/submission_xgb-no_outliers2.csv', index=False)